V:2022/04/04

Note: Pseudo code is shown in **bold** in the cell before each piece of code.

# `Lorentz_Lmfit.py`

The objective of this part of the process is to fit the amplitude spectrum, now free of anthropogenic noise, of each component (NS and EW) of the horizontal magnetic field, for each 10-min interval. Each spectrum, within the calibration band of the magnetometers, is fitted to a sum of three Lorentzian functions and a linear term. The justification of this process, the minimization method and the chosen parameters are explained in Rodríguez‐Camacho (2018). In this section we will focus on explaining the code and how to use the it.

The process consists of the following steps:
1. Loading the amplitude spectrum for each 10-min interval.
2. Defining the frequency band, within the calibrated frequency band, in which the Lorentzian fitting is to be performed, 
3. Choosing the initial parameter values.
3. Lorentzian fitting of the amplitude spectrum with the function `lmfit`.


The Lorentzian fit consists on minimizing the mean square error between the experimental spectrum and the function defined by:

$$ L(f)=\sum_{i=1}^{3}\frac{A_i}{1+\left( \frac{f-f_i}{\sigma_i} \right)^2}+B f+C $$

where $A_i$ stands for the amplitude of the $i$th mode, resonating at frequency $f_i$ and with a bandwidth $\sigma_i$, while the parameters $B$ and $C$ correspond to the stright line.

In addition to these 11 parameters, we can define a global mode amplitude, $ P_i $, as the value of the fitting curve in the resonance frequencies $ f_i $. Namely:

$$ P_i=L(f_i) $$

which defines a total of 14 parameters. 


The outputs of this code are, for each sensor:

- A list of 14 parameters for each 10-min interval of the month. They correspond to 3 global mode amplitudes, 3 amplitudes, 3 resonance frequencies, 3 resonance frequency widths and 2 parameters of the line (slope and intercept).
- A list of the 11 error estimates, each one associated with each parameter for each 10-min interval of the month.
- A list of the root mean square error of the fitting in each 10-min interval of the month.

There is a third group of resonance amplitudes and frequencies, local maximum amplitudes and local maximum frequencies, whose calculation is left to the next block code `empaque` in order to separate it from the fitting process itself, which makes it possible to leave it as optional.

**Python packages used in the program**

In [1]:
import numpy as np
from lmfit import minimize, Parameters

We use the packages `numpy`,` lmfit.minimize` and `lmfit.Parameters`. The `lmfit` module is not installed by default within Python. To install it from `pip` we use:

`pip install lmfit`

If the Python installation was done with Anaconda, we use the instruction:

`conda install -c conda-forge lmfit`

## Starting point

**Definition of paths and station parameters. The initial values for the parameters are also defined.**

In [2]:
year='2015'
month='1503'
pathg="S_N_FD/"

# Station parameters
nventa=2**13   # Number of samples in the FFT window
fm=256    # Sampling frequency
fcal_inf,fcal_sup= 6., 25.         # Calibrated frequency band

# Fitting parameters
fajus_inf,fajus_sup= 6.35, 23.75   # Fitting frequency band
nreso= 3                           # Number of modes
# Initial values
f1i, f2i, f3i= 8.011, 14.2, 20.63
s1i, s2i, s3i= 1.78, 1.94, 2.56
mi, ni= 0., 0.
metodo= 'leastsq' 

**Definition of several variables**

In [3]:
df=float(fm)/float(nventa)    # Frequency increment
npara= nreso*3+2           # Number of fitting parameters
def pofre(f,f0,df):
    return int(round((f-f0)/df))
fajus_inf_pos= pofre(fajus_inf,fcal_inf,df)
fajus_sup_pos= pofre(fajus_sup,fcal_inf,df) 
fre= np.arange(fcal_inf,fcal_sup+df,df)  
nf= len(fre)                                 
freajus= fre[fajus_inf_pos : fajus_sup_pos+1]  
nfajus= len(freajus)

The function `pofre (f, f0, df)` is defined, which is in charge of finding the position of the frequency `f` within a list that starts with the frequency `f0` and has an increment of `df` (it is considered that the first position is 0).

It is convenient to know the position of the first and last frequencies of the
fitting band within the list of frequencies in the calibration band, `fajus_inf_pos`and `fajus_sup_pos`. 

The number of resonances, `nreso = 3`, and the number of fitting parameters,` npara = nreso * 3 + 2` are also defined.

## Loading

**The 10-min amplitude spectra, free of anthropogenic noise, are loaded**

In [4]:
# Paths
pathmonth=pathg+year+"/"+month+"/"+"SR"+month+"_" 
# Files (output from antropo.py) are loaded
rs0= np.genfromtxt(pathmonth+"mediaNA"+'_0').reshape(-1,nf)
rs0_ajus= rs0[:,fajus_inf_pos : fajus_sup_pos+1]
rs1=np.genfromtxt(pathmonth+"mediaNA"+'_1').reshape(-1,nf)
rs1_ajus= rs1[:,fajus_inf_pos : fajus_sup_pos+1]
nintervmes=len(rs0)

The output of the previous code, `anthropo`, consisting of the spectra in the calibrated band after removing the anthropogenic noise, is loaded. We do it for each sensor. From these data, we calculate the number of intervals that the month contains, `nintervmes` (it is assumed that for both sensors there is the same number of intervals, so the definition is made for sensor 0).

We will work with the spectrum in the calibrated frequency band, `rs0` for sensor NS and `rs1` for sensor EW, and with the part of the spectrum in the fitting frequency band, `rs0_ajus` and `rs1_ajus` respectively. We will therefore have two data lengths, depending on whether we consider spectrum data in the calibrated frequency band or in the adjusted frequency band.

## Initial amplitude values estimation

**The initial amplitude values are defined from the monthly averaged amplitude spectrum.**

In [5]:
para_ini0= np.array([0.,0.,0.,f1i,f2i,f3i,s1i,s2i,s3i,mi,ni])
para_ini1= np.array([0.,0.,0.,f1i,f2i,f3i,s1i,s2i,s3i,mi,ni])
f1i_pos= pofre(f1i,fajus_inf,df)
f2i_pos= pofre(f2i,fajus_inf,df)
f3i_pos= pofre(f3i,fajus_inf,df)

rs0_ajus_mediaMes= np.mean(rs0_ajus,0)
rs1_ajus_mediaMes= np.mean(rs1_ajus,0)

para_ini0[0]= rs0_ajus_mediaMes[f1i_pos] 
para_ini0[1]= rs0_ajus_mediaMes[f2i_pos] 
para_ini0[2]= rs0_ajus_mediaMes[f3i_pos]
para_ini1[0]= rs1_ajus_mediaMes[f1i_pos] 
para_ini1[1]= rs1_ajus_mediaMes[f2i_pos] 
para_ini1[2]= rs1_ajus_mediaMes[f3i_pos]


The initial values for the amplitude fitting parameters are obtained from the average of the intervals of the month for each sensor (the result of these averages is stored in the arrays `rs0,1_ajus_mediaMes`) at the initial resonance frequency values.

The initial parameters of the fitting for the frequencies and the widths are taken from Toledo‐Redondo (2010).  The slope and intercept of the linear part are taken as zero:
 
`f1i, f2i, f3i= 8.011, 14.2, 20.63
 s1i, s2i, s3i= 1.78, 1.94, 2.56
 mi, ni= 0., 0.`

### Fitting function definitions

**The function `residual` defines the model and data difference.**

In [6]:
def residual(params, x, data):
    v= params.valuesdict()     
    model = v['a1']/(1+((x-v['f1'])**2)/v['s1']**2)+\
                v['a2']/(1+((x-v['f2'])**2)/v['s2']**2)+\
                v['a3']/(1+((x-v['f3'])**2)/v['s3']**2)+v['m']*x+v['n']      
    return (data-model)

The class `Parameters()` is used to define the `params` object that we will use in the program. This object has the structure of a dictionary.

The function `residual` defines the values to be minimized, which in our case will be defined by Lorentzian functions and a linear term.
The arguments are: the dictionary `params`, the frequency list `x`, and the values `data`. 
Firstly, the values for each parameter of the fitting function are stored in the list `v` through the `.valuesdict ()` method. The Lorentzian function is calculated for each value of the array `x` that must contain all the frequencies of the spectrum of the fitting band. Finally, the list with the difference between the fitting function and the measure is returned by the function and will be minimized with the function `lmfit` that will be called later.

This procedure is similar to that presented in the Lmfit package documentation: [Lmfit example](https://lmfit.github.io/lmfit-py/examples/example_fit_with_inequality.html#sphx-glr-examples-example-fit-with-inequality-py). 


**Function `defpar`: Fitting parameters definition. `leepar`, `leeerr`, `leechi`: Output arrays reading.**

In [7]:
def defpar(params,val):
    params.add('a1', value = val[0])
    params.add('a2', value = val[1])
    params.add('a3', value = val[2])    
    params.add('f1', value = val[3])
    params.add('f2', value = val[4])
    params.add('f3', value = val[5])    
    params.add('s1', value= val[6])
    params.add('s2', value= val[7])
    params.add('s3', value= val[8])    
    params.add('m', value= val[9])
    params.add('n', value= val[10])
# Output reading functions
def leepar(salida):
    val= np.zeros(11)
    val[0]=salida.params['a1'].value
    val[1]=salida.params['a2'].value
    val[2]=salida.params['a3'].value
    val[3]=salida.params['f1'].value
    val[4]=salida.params['f2'].value
    val[5]=salida.params['f3'].value    
    val[6]=salida.params['s1'].value
    val[7]=salida.params['s2'].value
    val[8]=salida.params['s3'].value
    val[9]=salida.params['m'].value
    val[10]=salida.params['n'].value
    return val
    
def leeerr(salida):
    val= np.zeros(11)
    val[0]=salida.params['a1'].stderr
    val[1]=salida.params['a2'].stderr
    val[2]=salida.params['a3'].stderr
    val[3]=salida.params['f1'].stderr
    val[4]=salida.params['f2'].stderr
    val[5]=salida.params['f3'].stderr   
    val[6]=salida.params['s1'].stderr
    val[7]=salida.params['s2'].stderr
    val[8]=salida.params['s3'].stderr
    val[9]=salida.params['m'].stderr
    val[10]=salida.params['n'].stderr
    return val
    
def leechi(salida):
    return salida.chisqr

This code defines fitting functions used together with the `lmfit` function.
The `defpar` function defines the dictionary `params` with the keys of each fitting parameter and the values `val`.

The function `leepar` extracts the values of the dictionary `params` calculated in the minimization process. The reading of the output for the error in each parameter, function `leeerr`, and the value of the funcion Chi-square,` leechi`, are also performed.

### Lorentzian fitting for each 10-min interval

**Definition of the output arrays**

In [8]:
params0 = Parameters()
params1 = Parameters()
defpar(params0, para_ini0)
defpar(params1, para_ini1)

para_s0= np.zeros((nintervmes, npara),dtype=float)
para_s1= np.zeros((nintervmes, npara),dtype=float)
salerr0=np.zeros((nintervmes, npara),dtype=float)
salerr1=np.zeros((nintervmes, npara),dtype=float)
salchi0=np.zeros((nintervmes),dtype=float)
salchi1=np.zeros((nintervmes),dtype=float)

This corresponds to the definition of the initial parameter values and the initialization of the arrays where the results will be stored.

**The function `minimize` de `lmfit` is called for each sensor and each 10-min interval. The outputs are saved.**

In [9]:
for i in np.arange(nintervmes):    
    # Sensor 0 
    out_s0 = minimize(residual, params0,\
            args=(freajus, rs0_ajus[i]), method = metodo)
    para_s0[i]= leepar(out_s0)
    salerr0[i]= leeerr(out_s0)
    salchi0[i]= leechi(out_s0)
    # Sensor 1    
    out_s1 = minimize(residual, params1,\
            args=(freajus, rs1_ajus[i]), method = metodo)
    para_s1[i]= leepar(out_s1)
    salerr1[i]= leeerr(out_s1)
    salchi1[i]= leechi(out_s1)

The `minimize` function is called and the corresponding outputs are read.

## Global mode amplitudes

**The sum of three lorentzian functions and the linear term is evaluated in each resonant frequency. These are the global mode amplitudes.**

In [10]:
salida0=np.zeros((nintervmes, npara+nreso),dtype=float)
salida1=np.zeros((nintervmes, npara+nreso),dtype=float)
salida0[:,nreso:]=para_s0[:,:]
salida1[:,nreso:]=para_s1[:,:]
# Global mode amplitudes are calculated from the global function
for i in np.arange(nintervmes):
    ampm0 = np.zeros(nreso)
    ampm1 = np.zeros(nreso)
    for k in np.arange(nreso):
        ampm0[k]=para_s0[i,nreso*3]*para_s0[i,nreso+k]+para_s0[i,nreso*3+1]
        ampm1[k]=para_s1[i,nreso*3]*para_s1[i,nreso+k]+para_s1[i,nreso*3+1]
        for l in np.arange(nreso):
            ampm0[k]+= para_s0[i,l]/(1+((para_s0[i,nreso+k]-\
                     para_s0[i,l+nreso])**2)/para_s0[i,l+2*nreso]**2)               
            ampm1[k]+= para_s1[i,l]/(1+((para_s1[i,nreso+k]-\
                     para_s1[i,l+nreso])**2)/para_s1[i,l+2*nreso]**2)   
    salida0[i,:nreso]=ampm0[:]
    salida1[i,:nreso]=ampm1[:]


The **global mode amplitudes** are the amplitudes corresponding to the values of the sum of the three Lorentzian functions plus the linear term, at each resonance frequency.

The global mode amplitudes are calculated for each 10-min interval of the month. The values of the three resonances and the linear terms are saved in the arrays `ampm0, ampm1`.

To save these values we define a new array for each sensor with three additional columns that we place at the beginning of the array. In these arrays, `salida0, salida1`, for sensor NS and EW, respectively, the parameters already calculated are stored.



## Outputs

**The output file paths are defined and the output arrays are written**

In [11]:
path0 = pathmonth+"mediaLO"+'_0'
path1 = pathmonth+"mediaLO"+'_1'
pathe0 = pathmonth+"mediaLOE"+'_0'
pathe1 = pathmonth+"mediaLOE"+'_1'
pathc0 = pathmonth+"mediaLOC"+'_0'
pathc1 = pathmonth+"mediaLOC"+'_1'

np.savetxt(path0,salida0.ravel())
np.savetxt(path1,salida1.ravel())
np.savetxt(pathe0,salerr0.ravel())
np.savetxt(pathe1,salerr1.ravel())            
np.savetxt(pathc0,salchi0.ravel())
np.savetxt(pathc1,salchi1.ravel())

Next, we proceed to define the output paths and file names and to write the parameters and errors in one column (for this, we use the `.ravel ()` method).

The output files therefore contain: the 14 fitting parameters for each 10-min interval (`mediaLO`), the error generated by the optimization algorithm for each parameter (`mediaLOE`) and the value of the chi-square function for the fitting curve with respect to the measured spectrum, also generated by the optimization algorithm (`mediaLOC`).

## References

Rodríguez‐Camacho, J., Fornieles, J., Carrión, M. C., Portí, J. A., Toledo‐Redondo, S., & Salinas, A. (2018). On the Need of a Unified Methodology for Processing Schumann Resonance Measurements. Journal of Geophysical Research: Atmospheres, 123(23), 13,277-13,290. https://doi.org/10.1029/2018JD029462

Toledo‐Redondo, S., A. Salinas, J. Portí, J. A. Morente, J. Fornieles, A. Méndez, J. Galindo‐Zaldívar, A. Pedrera, A. Ruiz‐Constán, and F. Anahnah (2010), Study of Schumann resonances based on magnetotelluric records from the western Mediterranean and Antarctica, J. Geophys. Res., 115, D22114, doi:10.1029/2010JD014316)